In [41]:
import requests
from bs4 import BeautifulSoup

link = "http://www.pyclass.com/real-estate/rock-springs-wy/LCWYROCKSPRINGS/"
link_header = {'User-agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:61.0) Gecko/20100101 Firefox/61.0'}

r = requests.get(link, headers=link_header)
c = r.content

soup = BeautifulSoup(c, "html.parser")

all = soup.find_all("div",{"class":"propertyRow"})

all[0].find("h4",{"class":"propPrice"}).text.replace("\n","").replace(" ","")

'$725,000'

In [42]:
l = []
for item in all :
    d = {}
    #find address in span
    d["Address"]=item.find_all("span",{"class":"propAddressCollapse"})[0].text 
    d["Locality"]=item.find_all("span",{"class":"propAddressCollapse"})[1].text
    d["Price"]=item.find("h4",{"class":"propPrice"}).text.replace("\n","").replace(" ","")
    try:
        d["Beds"]=item.find("span",{"class":"infoBed"}).find("b").text
    except:
        d["Beds"]=None
    try:
        d["Area"]=item.find("span",{"class":"infoSqFt"}).find("b").text
    except:
        d["Area"]=None
    try:
        d["Full Baths"]=item.find("span",{"class":"infoValueFullBath"}).find("b").text
    except:
        d["Full Baths"]=None
    try:
        d["Half Baths"]=item.find("span",{"class":"infoValueHalfBath"}).find("b").text
    except:
        d["Half Baths"]=None
    for column_group in item.find_all("div",{"class":"columnGroup"}):
        feature_groups = column_group.find_all("span",{"class":"featureGroup"})
        feature_names = column_group.find_all("span",{"class":"featureName"})
        for feature_group, feature_name in zip(feature_groups,feature_names):
            if "Lot Size:" in feature_group.text :
                d["Lot Size"]=feature_name.text

    l.append(d)

In [43]:
len(l)

10

In [44]:
import pandas
df = pandas.DataFrame(l)

In [45]:
df

,Address,Locality,Price,Beds,Area,Full Baths,Half Baths,Lot Size
0,0 Gateway,"Rock Springs, WY 82901","$725,000",None,None,None,None,NaN
1,1003 Winchester Blvd.,"Rock Springs, WY 82901","$452,900",4,None,4,None,0.21 Acres
2,600 Talladega,"Rock Springs, WY 82901","$396,900",5,"3,154",3,None,NaN
3,3239 Spearhead Way,"Rock Springs, WY 82901","$389,900",4,"3,076",3,1,"Under 1/2 Acre,"
4,522 Emerald Street,"Rock Springs, WY 82901","$254,000",3,"1,172",3,None,"Under 1/2 Acre,"
5,1302 Veteran's Drive,"Rock Springs, WY 82901","$252,900",4,"1,932",2,None,0.27 Acres
6,1021 Cypress Cir,"Rock Springs, WY 82901","$210,000",4,"1,676",3,None,"Under 1/2 Acre,"
7,913 Madison Dr,"Rock Springs, WY 82901","$209,000",3,"1,344",2,None,"Under 1/2 Acre,"
8,1344 Teton Street,"Rock Springs, WY 82901","$199,900",3,"1,920",2,None,"Under 1/2 Acre,"
9,4 Minnies Lane,"Rock Springs, WY 82901","$196,900",3,"1,664",2,None,2.02 Acres


In [46]:
df.to_csv("output.csv")

In [47]:
def list_per_page(soup) : 
    all = soup.find_all("div",{"class":"propertyRow"})
    l = []
    for item in all :
        d = {}
        #find address in span
        d["Address"]=item.find_all("span",{"class":"propAddressCollapse"})[0].text 
        d["Locality"]=item.find_all("span",{"class":"propAddressCollapse"})[1].text
        d["Price"]=item.find("h4",{"class":"propPrice"}).text.replace("\n","").replace(" ","")
        try:
            d["Beds"]=item.find("span",{"class":"infoBed"}).find("b").text
        except:
            d["Beds"]=None
        try:
            d["Area"]=item.find("span",{"class":"infoSqFt"}).find("b").text
        except:
            d["Area"]=None
        try:
            d["Full Baths"]=item.find("span",{"class":"infoValueFullBath"}).find("b").text
        except:
            d["Full Baths"]=None
        try:
            d["Half Baths"]=item.find("span",{"class":"infoValueHalfBath"}).find("b").text
        except:
            d["Half Baths"]=None
        for column_group in item.find_all("div",{"class":"columnGroup"}):
            feature_groups = column_group.find_all("span",{"class":"featureGroup"})
            feature_names = column_group.find_all("span",{"class":"featureName"})
            for feature_group, feature_name in zip(feature_groups,feature_names):
                if "Lot Size:" in feature_group.text :
                    d["Lot Size"]=feature_name.text
        l.append(d)
    return l

In [52]:
base_url = "http://www.pyclass.com/real-estate/rock-springs-wy/LCWYROCKSPRINGS/t=0&s="
result = []
r = requests.get(link, headers=link_header)
c = r.content
soup = BeautifulSoup(c, "html.parser")
page_number = soup.find_all("a",{"class":"Page"})[-1].text
for page in range(0,int(page_number)) :
    url = base_url+"%s.html"%(page*10)
    print(url)
    r = requests.get(url, headers=link_header)
    c = r.content

    soup = BeautifulSoup(c, "html.parser")
    result.extend(list_per_page(soup))
    print(len(result))
len(result)

http://www.pyclass.com/real-estate/rock-springs-wy/LCWYROCKSPRINGS/t=0&s=0.html
10
http://www.pyclass.com/real-estate/rock-springs-wy/LCWYROCKSPRINGS/t=0&s=10.html
20
http://www.pyclass.com/real-estate/rock-springs-wy/LCWYROCKSPRINGS/t=0&s=20.html
37


37

In [51]:
df = pandas.DataFrame(result)
df.to_csv("output.csv")